# Details to recreate the Plastisphere meta-analysis

This is a notebook that can be used to recreate all of the analyses found in the Plastisphere meta-analysis paper, or alternatively to re-do these analyses while also incorporating additional data.
Before running, you should check that:
- PICRUSt2 is installed and the environment is assumed to be called picrust2 (https://github.com/picrust/picrust2/wiki)
- R is installed (https://www.r-project.org/)
- You have downloaded all of the documents from https://doi.org/10.6084/m9.figshare.12227303:
    - *This contains all of the files produced during this analyses*
    - *If you wish to recreate the analyses, then take only the 'recreate_analyses' folder - the 'paper_data_20-04-14' may be used to compare your output with if you wish'*
    - *If you are also using your own data, then replace what is inside the qiime_output folder with your own data*
- You have installed the following Python packages: 
    - Bio (https://biopython.org/)
    - cartopy (https://scitools.org.uk/cartopy/docs/latest/)
    - pandas (https://pandas.pydata.org/)
    - scipy (https://www.scipy.org/)
    - scikit-bio (http://scikit-bio.org/)
    - scikit-learn (https://scikit-learn.org/stable/)
 - And the following R packages:
    - ggplot2 (https://ggplot2.tidyverse.org/)
    - phyloseq (https://joey711.github.io/phyloseq/)
    - ape (https://cran.r-project.org/web/packages/ape/ape.pdf)
    - microbiome (https://microbiome.github.io/tutorials/)
    - metacoder (https://grunwaldlab.github.io/metacoder_documentation/)
    - ggtree (https://guangchuangyu.github.io/software/ggtree/)
    - ggnewscale (https://github.com/eliocamp/ggnewscale)

It is expected that there are several other files in the directory that contains this notebook:
- agglomerate_and_unifrac.R
- all_functions_new.py
- metacoder.R
- metadata.txt
- plot_ancom_tree_heatmap.R
- Study_dates.csv
- Study_location.csv
- unifrac_grouped_samples.R
- world_map.jpg

In the folder 'recreate_analyses' you should have the following file setup (essential whether you are adding your own data or repeating the analyses in the paper) - this should be how it comes if you have downloaded from the above Figshare link:
- picrust
    - kegg_list.csv
    - ko_all.txt
- qiime_output
    - dna-sequences.fasta
    - feature-table_w_tax.txt
    - tree.nwk

If you want to save on computation time for recreating the analysis from the paper, then you can add the following folders/subfolders/files from the paper_data_20-04-14' folder (so as computationally intensive steps such as agglomeration and random forest construction are not carried out again):
- agglom (4 files)
    - otu_table.csv
    - reduced_tree.tree
    - unweighted_unifrac.csv
    - weighted_unifrac.csv
- random_forest  (7 files, 2 subfolders)
    - leave_one_dataset_out (7 files)
        - ASVs.csv
        - classes.csv
        - families.csv
        - genera.csv
        - orders.csv
        - phyla.csv
        - species.csv
    - single_environment (32 files)
        - aquatic.csv
        - freshwater.csv
        - marine.csv
        - terrestrial.csv
        The following for each of 'aquatic', 'freshwater', 'marine' and 'terrestrial', where the name in inverted commas replaces 'environment':
        - environment_ASVs.csv
        - environment_classes.csv
        - environment_families.csv
        - environment_genera.csv
        - environment_orders.csv
        - environment_phyla.csv
        - environment_species.csv
    - ASVs_overall.csv
    - classes_overall.csv
    - families_overall.csv
    - genera_overall.csv
    - orders_overall.csv
    - phyla_overall.csv
    - species_overall.csv
- picrust (4 files, 1 subfolder)
    - feature_table_agglom.txt
    - kegg_list.csv
    - ko_all.txt
    - sequences_agglom.fasta
    - picrust_out (3 files, 2 subfolders - although we only actually need one of these files from one subfolder)
        - ko_all_metagenome_out (4 files)
            - pred_metagenome_unstrat.tsv.gz

For any of the functions here, you can find out more information by using the doc strings, i.e. by typing print(function_name.__doc__)

1. First, import the functions and the other script that we will be using here:

In [1]:
import all_functions_new as af
import os
import pandas as pd
import pickle
from datetime import datetime
import importlib

2. Set up the names and locations of some files, and make some empty folders:
Note *You should change basedir to be wherever your folder is that contains picrust and qiime_output
You should also change n_jobs to be the number of processors that you want to use (this will affect the speed with which many functions run)*

In [3]:
basedir = '/Users/robynwright/Documents/OneDrive/ACU_meta-analysis/Data_2020/paper_data_20-04-14/' 
ft_tax, meta_fn, seqs, study_locs, study_dates = basedir+'qiime_output/feature-table_w_tax.txt', 'metadata.txt', basedir+'qiime_output/dna-sequences.fasta', 'Study_location.csv', 'Study_dates.csv'
n_jobs, est = 10, 1

folder_names = ["agglom", "picrust", "figures", "ancom", "figures/ancom", "figures/metacoder", "random_forest", "random_forest/single_environment", "random_forest/leave_one_dataset_out", "figures/random_forest", "figures/random_forest/single_environment", "figures/random_forest/leave_one_dataset_out"]
for fn in folder_names:
    if not os.path.exists(basedir+"/"+fn):
       os.system("mkdir "+basedir+"/"+fn)

3. Now format the QIIME2 output files for running the R agglomeration and unifrac scripts
Note *If you already have the R input and tax_dict made then to save time, you can just open the tax_dict*

In [17]:
if os.path.exists(basedir+'tax_dict.dictionary'):
    tax_dict = af.open_pickle(basedir+'tax_dict.dictionary')
else:
    ft, tax_dict = af.format_R(ft_tax, basedir)

4. Run the R script:
Note *the location here may need changing if this doesn't work. You can type which Rscript into terminal to find out the output here*

In [5]:
if not os.path.exists(basedir+'agglom/otu_table.csv'): #if we haven't already done this and got the agglomerated OTU table
    os.rename(basedir+'feature_table.csv', 'feature_table.csv') #move the feature table to the right directory
    os.rename(basedir+'qiime_output/tree.nwk', 'tree.nwk') #and the tree
    
    #Change the location of Rscript, if necessary
    os.system("/usr/local/bin/Rscript agglomerate_and_unifrac.R") #run the R script to agglomerate and calculate unifrac distances
    
    os.rename('feature_table.csv', basedir+'feature_table.csv') #now move the feature table back to where it came from
    os.rename('tree.nwk', basedir+'qiime_output/tree.nwk') #and the tree
    os.rename('otu_table.csv', basedir+'agglom/otu_table.csv') #move the OTU table to the right directory
    os.rename('reduced_tree.tree', basedir+'agglom/reduced_tree.tree') #and the tree
    os.rename('weighted_unifrac.csv', basedir+'agglom/weighted_unifrac.csv') #and the weighted unifrac distance matrix
    os.rename('unweighted_unifrac.csv', basedir+'agglom/unweighted_unifrac.csv') #and the unweighted unifrac distance matrix
    

5. Now we can read in these new files and open the information that we need:

In [6]:
ft_df, tree_agglom = af.open_and_sort(basedir+'/agglom/otu_table.csv'), basedir+'/agglom/reduced_tree.tree'
w_uf, uw_uf = af.open_and_sort(basedir+'agglom/weighted_unifrac.csv'), af.open_and_sort(basedir+ 'agglom/unweighted_unifrac.csv') #file names for unifrac distances based on agglomerated data
meta, meta_names, meta_dict = af.get_meta(meta_fn)
meta_df = af.get_meta_df(meta, meta_names, list(ft_df.columns))
ft_full = basedir+'feature_table.csv'

if os.path.exists(basedir+'ASV_dict.dictionary'):
    ASV_dict = af.open_pickle(basedir+'ASV_dict.dictionary')
else:
    ASV_dict = af.get_ASV_dict(ft_df, basedir+seqs)
    af.write_pickle(basedir+'ASV_dict.dictionary', ASV_dict)

6. PICRUSt (only run if we don't have this already)

In [7]:
if not os.path.exists(basedir+'picrust/picrust_out/ko_all_metagenome_out/pred_metagenome_unstrat.tsv.gz'):
    seqs_agglom_fn, ft_agglom_fn = af.filter_seqs(ft_df, seqs) 
    os.system("source activate picrust2") 
    os.system("picrust2_pipeline.py -s "+seqs_agglom_fn+" -i "+ft_agglom_fn+" -o picrust/picrust_out --custom_trait_tables ko_all.txt --stratified --no_pathways") 
if not os.path.exists(basedir+'KO_dict.dictionary'):
    ko_file = basedir+'picrust/kegg_list.csv' 
    KO_dict, KO_dict_full = af.make_KO_dict(ko_file) 
else:
    KO_dict = af.open_pickle(basedir+'KO_dict.dictionary') 
if not os.path.exists(basedir+'picrust.dataframe'):
    picrust_file = 'picrust/picrust_out/ko_all_metagenome_out/pred_metagenome_unstrat.tsv.gz' 
    picrust = pd.read_csv(basedir+'/'+picrust_file, sep='\t', header=0, index_col=0) 
    picrust, KO_dict = af.filter_picrust(basedir+picrust_file, KO_dict, KO_dict_full) 
else:
    picrust = af.open_pickle(basedir+'picrust.dataframe')

Now we are ready to do all of the analysis and make all of the plots

7. Get basic study map and metrics:

In [8]:
af.study_map(study_dates, study_locs, basedir) 
af.study_metrics(meta_df, basedir)

8. Get the similarity heatmap and nMDS plots using the weighted and unweighted unifrac distances

In [12]:
w_uf.to_csv(basedir+'agglom/sorted_weighted_unifrac.csv')
uw_uf.to_csv(basedir+'agglom/sorted_unweighted_unifrac.csv')
af.nmds_plot_study_env('agglom/sorted_weighted_unifrac.csv', 'agglom/sorted_unweighted_unifrac.csv', meta_dict, basedir) 
af.similarity_heatmap_combined(w_uf, uw_uf, basedir) 

/Users/robynwright/Documents/OneDrive/Github/Plastisphere-MetaAnalysis/python_analysis_20-04-14/all_functions_new.py:824: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if npos == '':
/Users/robynwright/Documents/OneDrive/Github/Plastisphere-MetaAnalysis/python_analysis_20-04-14/all_functions_new.py:824: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if npos == '':


9. Get the figure that summarises the sample types, groups them to a dendrogram, gets stacked bar plots at phyla level, heatmap of families, simpsons diversity and venn diagram of ASVs shared between sample types in different environments

In [13]:
af.bar_dendro_venn(ft_df, ft_full, meta_dict, basedir, tax_dict)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


10. Get Ancom trees and heatmaps for early and late incubation times

In [ ]:
[af.tree_heatmap(ft_df, meta_dict, basedir, tax_dict, level=al) for al in [1, 2, 3, 4, 5, 6]]

11. Get the metacoder plots for early and late incubation times

In [ ]:
af.metacoder(ft_df, tax_dict, meta_dict, basedir) 

12. Get the overall random forests (skip this if you downloaded the files of the already constructed random forests)
*Note that if you re-run this section then you might get slightly different results than the paper due to different random selections of 20%/80% of the data for testing/training)*

In [ ]:
af.get_random_forests(ft_df, tax_dict, meta_df, basedir, est=est, n_jobs=n_jobs)

13. Now check how the accuracy of our models goes down when we leave out individual studies (skip this if you downloaded the files of the already constructed random forests)

In [ ]:
af.get_random_forests_leave_one_dataset_out(ft_df, tax_dict, meta_df, basedir, meta_dict, est=10000, n_jobs=n_jobs)

14. Get the random forests for each environment for general plastic type

In [ ]:
af.get_environment_random_forest(ft_df, tax_dict, meta_df, meta_dict, basedir, est=10000, n_jobs=10) 

15. And get the random forest figures

In [18]:
#af.get_random_forest_plots(ft_df, tax_dict, ASV_dict, meta_dict, basedir)
#af.leave_one_out_plots(basedir, meta_df)
af.get_overall_random_forest_plot(ft_df, meta_df, tax_dict, ASV_dict, meta_dict, basedir)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


KeyError: 'Thermotogae'

16. And get the environment random forest figures for general plastic type

In [ ]:
af.get_environment_random_forest_plots(ft_df, meta_df, tax_dict, ASV_dict, meta_dict, basedir)
af.make_env_rf_plot(ft_df, tax_dict, basedir, ASV_dict, meta_dict)
af.make_env_rf_plot(ft_df, tax_dict, basedir, ASV_dict, meta_dict, mx=0.01)

17. Get the PICRUSt2 plots

In [ ]:
KO_dict = af.open_pickle(KO_dict) #load the picrust kegg ortholog dictionary 
picrust = af.open_pickle(picrust) #and the picrust data
af.picrust_plots(picrust, KO_dict, meta_dict, basedir) #now make the picrust volcano and bar plot

18. And get the separate plots for each study

In [ ]:
af.plots_per_study(ft_df, meta_df, meta_dict, w_uf, uw_uf, tax_dict, ASV_dict, basedir, est=est, n_jobs=n_jobs) #get the plots summarising the results per study

19. And check how long everything took to run (if desired)

In [ ]:
print('Running time was', datetime.now()-start_time)